In [37]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
 
df = pd.read_excel('MVO Assignment.xlsx')
tickers = df['Unnamed: 1'].dropna()
param = list(df['Unnamed: 4'].dropna())

tickers


1    AAPL
2     IBM
3     MCD
4     PFE
5     WMT
Name: Unnamed: 1, dtype: object

In [42]:
import yfinance as yf

history = pd.DataFrame()

for ticker in tickers:
    yf_ticker = yf.Ticker(ticker)
    #history[ticker] = yf_ticker.history(period=param[0], interval = param[1], auto_adjust = True)['Close']
    history[ticker] = yf_ticker.history(interval=param[1], start = '2015-11-01',end='2023-01-01', auto_adjust = True)['Close']

    col_shift = ticker + '_shifted'
    history[col_shift] = history[ticker].shift(periods = 1,fill_value = 0, axis = 0)

    col_ret = ticker + '_returns'
    history[col_ret] = (history[ticker] - history[col_shift])/history[col_shift]

    history = history.drop(columns=[col_shift, ticker])


history = history.tail(-1)
mu_vector = []
for i,ticker in enumerate(tickers):
    col_ret = ticker + '_returns'
    mean = history[col_ret].mean()
    col_de_mean = ticker + '_de_meaned'
    history[col_de_mean] = history[col_ret] - mean
    history = history.drop(columns=[col_ret])

    mu_vector.append(mean) 
    
history.index = history.index.tz_localize(None)

print(mu_vector)
print(history)



[0.022379782124814528, 0.00699766849504804, 0.013322043938443261, 0.011170447555653495, 0.013658635741694989]
            AAPL_de_meaned  IBM_de_meaned  MCD_de_meaned  PFE_de_meaned  \
Date                                                                      
2015-12-01       -0.128800      -0.010649       0.029655      -0.018172   
2016-01-01       -0.097622      -0.100225       0.034418      -0.066623   
2016-02-01       -0.029057       0.043006      -0.066562      -0.038064   
2016-03-01        0.110947       0.160651       0.067249      -0.002138   
2016-04-01       -0.162301      -0.043379      -0.006877       0.092406   
...                    ...            ...            ...            ...   
2022-08-01       -0.054931      -0.024889      -0.055430      -0.108756   
2022-09-01       -0.142136      -0.070393      -0.093697      -0.043671   
2022-10-01        0.087172       0.156961       0.168354       0.052586   
2022-11-01       -0.057009       0.069725      -0.012845       0.

In [43]:
import numpy as np

# (Epsilon transpose * Epsilon )/ (n-1)
covariance = np.matmul(np.transpose(history),history)/(len(history) - 1)

covariance

,0,1,2,3,4
AAPL_de_meaned,0.007576,0.002006,0.001559,0.001370,0.001556
IBM_de_meaned,0.002006,0.005065,0.001549,0.001740,0.000676
MCD_de_meaned,0.001559,0.001549,0.002598,0.001050,0.001239
PFE_de_meaned,0.001370,0.001740,0.001050,0.004428,0.000874
WMT_de_meaned,0.001556,0.000676,0.001239,0.000874,0.002950


In [6]:
import openpyxl
wb = openpyxl.load_workbook('MVO Assignment.xlsx', data_only=True)
sheet = wb['Returns']

In [7]:
sheet.delete_cols(1, 100000)
sheet.delete_rows(1, 100000)


columns = history.columns.values.tolist()
sheet.cell(column=1, row=1).value = 'Time'



for i, time in enumerate(history.index.values.tolist()):
    if i==0:
        pass
    else:
        sheet.cell(column=1, row=i+1).value = pd.to_datetime(time)


for i, column in enumerate(columns):
    
    for j, value in enumerate(history[column].tolist()):
        sheet.cell(column=i+2, row=j+1).value = value
        
for i, column in enumerate(columns):
    sheet.cell(column=i+2, row=1).value = column

wb.save('MVO Assignment.xlsx')

/var/folders/hg/kqg69xz568l28vnvcx6x09hh0000gn/T/ipykernel_35751/4165443610.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  covariance = np.matmul(np.transpose(history.tail(-1)),history.tail(-1))/(len(history.tail(-1)) - 1)
